In [ ]:
# SETUP

# Install our additional project requirements
%pip install --quiet --no-index --find-links=/kaggle/input/mammography-train/pip -Ur /kaggle/input/mammography-train/kaggle/mammography/requirements/main.in

In [ ]:
!cp /kaggle/input/mammography-train/checkpoints /kaggle/working/checkpoints
!cp /kaggle/input/mammography-train/wandb /kaggle/working/wandb

In [ ]:
# Print system info
# !lscpu
# !nvidia-smi
# !python -m torch.utils.collect_env

In [ ]:
# Submit
%env PYTHONPATH /kaggle/input/mammography-train/kaggle/
!python -m mammography.src.submit \
    trainer.default_root_dir=/kaggle/working/ \
    datamodule.metadata_filepath=/kaggle/input/rsna-breast-cancer-detection/test.csv \
    datamodule.image_dir=/kaggle/input/rsna-breast-cancer-detection/test_images \
    +trainer.accelerator=gpu \
    datamodule.checkpoint_path=/kaggle/working/checkpoints/last.ckpt \
    model.checkpoint_path=/kaggle/working/checkpoints/last.ckpt